#### 1. Exploratory Data Anyalysis

#### 2. 데이터셋 출처
- Pima Indians Diabetes Database|Kaggle

#### 2.1 데이터 구성
- Pregnancies: 임신 횟수
- Glucose: 2시간 동안의 경구 포도당 내성 검사에서 혈장 포도당 농도
- BloodPressure:이완기 혈압 (mm Hg)
- SkinThickness: SkinThicknessTriceps 피부 주름 두께(mm)
- Insulin:2시간 혈청 인슐린 (mu U/ml)
- BMI: 체질량 지수(체중kg/ 키(m)^2)
- DiabetesPedigreeFunction: 당뇨병 혈통 기능
- Age: 나이
- Outcome: 768개 중에 268개의 결과 클래스 변수(0 또는 1)는 1이고 나머지는 0입니다.

#### 라이브러리 로드

In [ ]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy
# 시각화를 위한 seaborn, matplotlib.pyplot 을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#### 4 데이터 로드

In [ ]:
df=pd.read_csv('data/diabetes.csv')
df.shape

In [ ]:
# 위에서 5개만 미리보기 합니다.
df.head()

In [ ]:
# info로 데이터타입, 결측치, 메모리 사용량 등의 정보를 봅니다.
df.info() # 결측치가 없음

In [ ]:
# 결측치를 봅니다.
df_null=df.isnull()
df_null.head()

In [ ]:
df_null.sum()

In [ ]:
# 수치데이터에 대한 요약을 봅니다.
# 1사분위 값: 25%
# 2사분위 값: 50%
# 3사분위 값: 75%
# Gluecose에서 최솟값이 0인데 1사분위 값이 99이므로 결측치로 볼 수 있음
# BloodPressure(혈압)의 최솟값도 0이므로 결측치로 예상
# SkinThickness(피부두께)의 최솟값도 0이므로 결측치로 예상
df.describe()

In [ ]:
# 가장 마지막에 있는 Outcome은 label 값이기 때문에 제외하고
# 학습과 예측에 사용할 컬럼을 만들어 줍니다.
# feature_columns 라는 변수에 담아줍니다.

feature_columns=df.columns[:-1].tolist() # Age까지 가져오겠다.
feature_columns

#### 5 결측치 시각화

값을 요약해 보면 최솟값이 0으로 나오는 값들이 있습니다.
0이 나올 수 있는 값도 있지만 인슐린이나 혈압 등의 값은 0값이 결측치라고 볼 수 있을 것입니다.
따라서 0인 값을 결측치로 처리하고 시각화 해봅니다.

In [ ]:
cols=feature_columns[1:]
cols

In [ ]:
# 결측치 여부를 나타내는 데이터프레임을 만듭니다.
# 0값을 결측치라 가정하고 정답(label, target)값을 제외한 컬럼에 대해
# 결측치 여부를 구해서 df_null 이라는 데이터프레임에 담습니다.
# insulin 수치는 결측치가 너무 많기 때문에 당뇨병을 결정하는데 중요한 column이 아닐 것으로 예상
df_null=df[cols].replace(0,np.nan)
df_null=df_null.isnull()
df_null.sum()

In [ ]:
df_null.mean()*100

In [ ]:
# 결측치의 갯수를 구해 막대 그래프로 시각화 합니다.
df_null.sum().plot.barh()

In [ ]:
# 결측치를 heatmap으로 시각화 합니다.
plt.figure(figsize=(15,4))
sns.heatmap(df_null) # True는 검정색, False는 흰색

#### 6 정답값
- target, label이라고 부르기도 합니다.

In [ ]:
# 정답값인 outcome의 갯수를 봅니다.
df["Outcome"].value_counts()

In [ ]:
# 정답값인 outcome의 비율을 봅니다.
df["Outcome"].value_counts(normalize=True)*100

In [ ]:
# 다른 변수와 함께 봅니다.
# 임신횟수와 정답값을 비교해 봅니다.
# "Pregnancies"를 groupby로 그룹화해서 Outcome에 대한 비율을 구합니다.
# 결과를 df_po라는 변수에 저장합니다.

df_po=df.groupby(["Pregnancies"])["Outcome"].agg(["mean","count"]).reset_index()
df_po

In [ ]:
# 임신횟수에 따른 당뇨병 발병 비율
df_po["mean"].plot.bar(rot=0)

#### 7 countplot

In [ ]:
# 위에서 구했던 당뇨병 발병 비율을 구해봅니다.
# 당뇨병 발병 빈도수를 비교 합니다.

sns.countplot(data=df,x="Outcome")

In [ ]:
# 임신횟수에 따른 당뇨병 발병 빈도수를 비교합니다.

sns.countplot(data=df,x="Pregnancies",hue="Outcome")

In [ ]:
# 임신횟수의 많고 적음에 따라 Pregnancies_high 변수를 만듭니다.
df["Pregnancies_high"]=df["Pregnancies"] > 6
df[["Pregnancies_high","Pregnancies"]].head()

In [ ]:
# Pregnancies_high 변수의 빈도수를 countplot으로 그리고
# Outcome 값에 따라 다른 색상으로 표현합니다.

sns.countplot(data=df, x="Pregnancies_high", hue="Outcome")

#### 8 barplot
- 기본 설정으로 시각화 하면 y축에는 평균을 추정해서 그리게 됩니다.


In [ ]:
# 당뇨병 발병에 따른 BMI 수치를 비교합니다.
sns.barplot(data=df,x="Outcome",y="BMI")

In [ ]:
# 당뇨병 발병에 따른 포도당(Gluecose) 수치를 비교합니다.
sns.barplot(data=df,x="Outcome",y="Glucose")

In [ ]:
# Insuline 수치가 0 이상인 관측치에 대해서 당뇨병 발병을 비교합니다.
sns.barplot(data=df, x="Outcome",y="Insulin")

In [ ]:
# 임신횟수에 대해서 당뇨병 발병 비율을 비교합니다.
sns.barplot(data=df,x="Pregnancies",y="Outcome")

In [ ]:
# 임신횟수(Pregnancies)에 따른 포도당(Gluecose)수치를 당뇨병 발병여부(Outcome)에 따라 시각화 합니다.
sns.barplot(data=df,x="Pregnancies",y="Glucose",hue="Outcome")

In [ ]:
# 임신횟수(Pregnancies)에 따른 체질량지수(BMI)를 당뇨병 발병여부(Outcome)에 따라 시각화 합니다.
sns.barplot(data=df,x="Pregnancies",y="BMI",hue="Outcome")

In [ ]:
# 임신횟수(Pregnancies)에 따른 인슐린 수치(Insulin)를 당뇨병 발병여부(Outcome)에 따라 시각화 합니다.
# 인슐린 수치에는 결측치가 많기 때문에 0보다 큰 값에 대해서만 그립니다.
sns.barplot(data=df[df["Insulin"]>0],x="Pregnancies",y="Insulin",hue="Outcome")

#### 9 boxplot

In [ ]:
# 임신횟수 (Pregnancies)에 따른 인슐린 수치(Insulin)를 당뇨병 발병여부(Outcome)에 따라 시각화 합니다.
# 인슐린 수치에는 결측치가 많기 떄문에 0보다 큰 값에 대해서만 그립니다.

sns.boxplot(data=df[df["Insulin"]>0], x="Pregnancies",y="Insulin",hue="Outcome")

#### 10 violinplot

In [ ]:
plt.figure(figsize=(20, 5))
sns.violinplot(data=df[df["Insulin"] > 0], x="Pregnancies", y="Insulin", hue="Outcome", split=True)

#### swarmplot

In [ ]:
plt.figure(figsize=(15, 4))
sns.violinplot(data=df[df["Insulin"] > 0], x="Pregnancies", y="Insulin", hue="Outcome")